In [3]:
import pandas as pd
import pandas as pd
import requests

In [4]:
df = pd.read_excel("ATLCodeViolationComplaints_Nov2009_Sept2023.xlsx")


In [8]:
df.head()

,Date,ComplaintNumber,ComplaintType,Description,Address,Status,ShortNotes,Year,Month,Day
0,2023-09-01,CC-2023-04461,Code Complaint,Apt 3454.. roach infestation,"855 EMORY POINT DR NE, APT 3454, ATLANTA GA 30329",Citation Initiated,Highly Hazardous,2023,9,1
1,2023-09-01,CC-2023-04460,Code Complaint,Apt 1417.. mold due to active water leak from ac,"565 MCDANIEL ST SW, ATLANTA GA 30312",No Violation Found,Highly Hazardous,2023,9,1
2,2023-09-01,CC-2023-04459,Code Complaint,Caller states low pressure and now none at all...,"1270 SPRING ST NW, ATLANTA GA 30309",Unable to Gain Entry,Highly Hazardous,2023,9,1
3,2023-09-01,CC-2023-04458,Code Complaint,"Overgrowth, junk vehicles, junk tires, exterio...","3155 HICKMAN DR NW, ATLANTA GA 30311",Research Return to Clerical,Property Maintenance,2023,9,1
4,2023-09-01,CC-2023-04457,Code Complaint,Junk Trash Debris in the parking lot.,"1001 PONCE DE LEON AVE NE, ATLANTA GA 30306",No Violation Found,Property Maintenance,2023,9,1


In [10]:
df.isna()

,Date,ComplaintNumber,ComplaintType,Description,Address,Status,ShortNotes,Year,Month,Day
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
113472,False,False,False,True,False,False,False,False,False,False
113473,False,False,False,True,False,False,False,False,False,False
113474,False,False,False,True,False,False,False,False,False,False
113475,False,False,False,True,False,False,False,False,False,False


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113477 entries, 0 to 113476
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Date             113477 non-null  datetime64[ns]
 1   ComplaintNumber  113474 non-null  object        
 2   ComplaintType    113477 non-null  object        
 3   Description      102808 non-null  object        
 4   Address          113459 non-null  object        
 5   Status           113475 non-null  object        
 6   ShortNotes       108640 non-null  object        
 7   Year             113477 non-null  int64         
 8   Month            113477 non-null  int64         
 9   Day              113477 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 8.7+ MB


In [6]:
session = requests.Session()

def geocode_address(session, input_address):
    geocoding_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": input_address,
        "format": "json",
    }
    
    response = session.get(geocoding_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            first_result = data[0]
            return float(first_result["lon"]), float(first_result["lat"])
    
    # Log failed geocoding requests
    print(f"Geocoding failed for address: {input_address}")
    return None, None

# Assuming 'df' is your DataFrame with an 'Address' column
df['locations'] = df['Address'].head(10).apply(lambda address: geocode_address(session, address))

# Create a new DataFrame to store the geocoding results
geocoded_df = pd.DataFrame(df['locations'].head(10).tolist(), columns=['lon', 'lat'])
geocoded_df['Address'] = df['Address'].head(10)  # Add the original addresses for reference
display(geocoded_df)



Geocoding failed for address: 855 EMORY POINT DR NE, APT 3454, ATLANTA GA 30329


,lon,lat,Address
0,NaN,NaN,"855 EMORY POINT DR NE, APT 3454, ATLANTA GA 30329"
1,-84.402379,33.739832,"565 MCDANIEL ST SW, ATLANTA GA 30312"
2,-84.389840,33.789407,"1270 SPRING ST NW, ATLANTA GA 30309"
3,-84.490138,33.755072,"3155 HICKMAN DR NW, ATLANTA GA 30311"
4,-84.354644,33.773179,"1001 PONCE DE LEON AVE NE, ATLANTA GA 30306"
5,-84.527025,33.744062,"375 DOLLAR MILL RD SW, ATLANTA GA 30331"
6,-84.475494,33.744311,"2694 LISA DR SW, ATLANTA GA 30311"
7,-84.458796,33.769085,"509 ABERDEEN DR NW, ATLANTA GA 30318"
8,-84.339577,33.743883,"1469 MCPHERSON AVE SE, ATLANTA GA 30316"
9,-84.495953,33.717523,"1355 CASCADE FALLS DR SW, ATLANTA GA 30311"


In [3]:
import pandas as pd
import requests

# Create a session for making HTTP requests (reusable)
session = requests.Session()

def geocode_address(session, input_address):
    geocoding_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": input_address,
        "format": "json",
    }
    
    response = session.get(geocoding_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            first_result = data[0]
            return float(first_result["lon"]), float(first_result["lat"])
    
    return None, None

batch_size = 10000
num_batches = len(df) // batch_size + 1

geocoded_results = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch_df = df.iloc[start_idx:end_idx]
    
    # Apply geocoding to the current batch of addresses
    batch_df['locations'] = batch_df['Address'].apply(lambda address: geocode_address(session, address))
    
    # Append the geocoded results to the list
    geocoded_results.append(batch_df)

# Concatenate the geocoded results from all batches into a single DataFrame
geocoded_df = pd.concat(geocoded_results, ignore_index=True)
print(geocoded_df)


KeyboardInterrupt: 